# ReAct Agent

In this notebook we will look into creating ReAct Agent over tools.

1. ReAct Agent over simple calculator tools.
2. ReAct Agent over QueryEngine (RAG) tools.

### Installation

In [21]:
!pip install llama-index
!pip install llama-index-llms-anthropic
!pip install llama-index-embeddings-huggingface

### Setup API Keys

In [22]:
from google.colab import userdata
ANTHROPIC_API_KEY=userdata.get('ANTHROPIC_API_KEY')

In [23]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

import os

# Using Anthropic LLM API for LLM
os.environ['ANTHROPIC_API_KEY'] = ANTHROPIC_API_KEY

from IPython.display import display, HTML

### Set LLM and Embedding model

We will use anthropic latest released `Claude-3 Opus` LLM.

In [24]:
from llama_index.llms.anthropic import Anthropic
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [25]:
llm = Anthropic(temperature=0.0, model='claude-3-opus-20240229')
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [26]:
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

## ReAct Agent over Tools

### Define Tools

In [27]:
from llama_index.core.agent import ReActAgent
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

In [28]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)
multiply_tool = FunctionTool.from_defaults(fn=multiply)

### Create ReAct Agent

Create agent over tools and test out queries

In [29]:
agent = ReActAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)

In [30]:
response = agent.chat("What is 20+(2*4)? Calculate step by step ")

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
Thought: I now have the result of 2*4, which is 8. I need to add this result to 20 to get the final answer.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: To calculate 20+(2*4) step by step:

1. First, calculate 2*4 = 8
2. Then, add the result to 20: 20 + 8 = 28

Therefore, 20+(2*4) = 28.


In [31]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

### Visit Prompts

You can check prompts that the agent used to select the tools.

In [32]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: agent_worker:system_prompt

Value: 
You are designed to help with a variety of tasks, from answering questions     to providing summaries to other types of analyses.

## Tools
You have access to a wide variety of tools. You are responsible for using
the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools
to complete each subtask.

You have access to the following tools:
{tool_desc}

## Output Format
Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do

## ReAct Agent over `QueryEngine` Tools

In [33]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

### Download data

We will define ReAct agent over tools created on QueryEngines with Uber and Lyft 10K SEC Filings.

In [34]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-03-16 12:18:10--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.04s   

2024-03-16 12:18:10 (43.8 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-03-16 12:18:10--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP 

### Load Data

In [35]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

lyft_docs = SimpleDirectoryReader(input_files=["./data/10k/lyft_2021.pdf"]).load_data()
uber_docs = SimpleDirectoryReader(input_files=["./data/10k/uber_2021.pdf"]).load_data()

### Build Index

In [36]:
lyft_index = VectorStoreIndex.from_documents(lyft_docs)
uber_index = VectorStoreIndex.from_documents(uber_docs)

### Create QueryEngines

In [37]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

#### Create QueryEngine Tools

In [38]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

### ReAct Agent

In [39]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
)

### Querying with ReAct Agent

In [40]:
response = agent.chat("What was Lyft's revenue growth in 2021?")

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: According to the context provided, Lyft's revenue increased by $843.6 million, or 36%, in 2021 compared to 2020. This growth was primarily driven by a significant increase in the number of Active Riders in 2021 as vaccines became more widely distributed and more communities reopened. The revenue growth in 2021 was partially offset by increased driver incentives recorded as a reduction to revenue.
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Lyft's revenue grew by $843.6 million in 2021, which represents a 36% increase compared to 2020. The main driver of this revenue growth was a substantial increase in the number of Active Riders in 2021, as COVID-19 vaccines became more widely available and communities started reopening. However,

In [41]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

In [42]:
response = agent.chat(
    "Compare and contrast the revenue growth of Uber and Lyft in 2021, then"
    " give an analysis"
)


Thought: The current language of the user is: English. I need to use tools to find information about Uber and Lyft's revenue growth in 2021 to compare and contrast them.
Action: uber_10k
Action Input: {'input': "What was Uber's revenue growth in 2021?"}
Observation: According to the context provided, Uber's total revenue grew 57% year-over-year in 2021 to reach $17.5 billion. The revenue growth was driven by:

- Overall growth in Uber's Delivery business
- An increase in Freight revenue, attributable to the acquisition of Transplace in Q4 2021 as well as growth in the number of shippers and carriers on the network combined with an increase in volumes

Specifically, the revenue breakdown by segment shows:

- Mobility revenue grew from $6.1 billion in 2020 to $7.0 billion in 2021  
- Delivery revenue more than doubled from $3.9 billion in 2020 to $8.4 billion in 2021
- Freight revenue also doubled from $1.0 billion in 2020 to $2.1 billion in 2021

So in summary, Uber saw very strong 57% 

In [43]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))